In [1]:
import numpy as np
from src.cfg import CfgDataset,CfgAnglesNames,CfgBeamsNames
# from tabulate import tabulate

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Input, BatchNormalization, ReLU, MaxPool2D, UpSampling2D

from tensorflow.keras import backend
from tensorflow.keras.applications import imagenet_utils

from tensorflow.keras import layers
from tensorflow.python.util.tf_export import keras_export
from tensorflow.python.keras.utils import data_utils
import numpy as np
from matplotlib import pyplot as plt
import math

In [ ]:
step=5
folder = 'data/saved np'

xy_gauss_data=np.load(f'{folder}/'+CfgAnglesNames.approx_data+f'{step}.npy', allow_pickle=True)
xy_linear_data=np.load(f'{folder}/'+CfgBeamsNames.approx_data+f'{step}.npy', allow_pickle=True)

In [ ]:
texts_angles=np.load(f'{folder}/'+CfgAnglesNames.legend+f'{step}.npy', allow_pickle=True)
texts_beams=np.load(f'{folder}/'+CfgBeamsNames.legend+f'{step}.npy', allow_pickle=True)

In [ ]:
image_shape = (1024, 1024, 3)
model = tf.keras.applications.VGG19(include_top=False, weights='imagenet', input_shape=image_shape)
model.summary()

In [ ]:
for i, layer in enumerate(model.layers):
    print(f'i={i} name {layer.name}')

In [ ]:
all_images = np.load('data/saved np/all_images_no_preprocess.npy', allow_pickle=True)
all_images_rgb = []
for i, images_list in enumerate(all_images):
    for image_gray in images_list:
        tf_image = tf.expand_dims(image_gray / 255, 2)
        tf_rgb = tf.image.grayscale_to_rgb(tf_image)
        tf_preproc = tf.image.resize(tf_rgb, (1024, 1024))
        all_images_rgb.append(tf_preproc)

all_images_rgb = np.array(all_images_rgb)

In [ ]:
image=all_images_rgb[0]

layers_name = ['block1_pool', 'block2_pool', 'block3_pool', 'block4_pool', 'block5_pool']
outputs = [model.get_layer(name).output for name in layers_name]

intermediate_layer_model = tf.keras.Model(inputs=model.input,
                                          outputs=outputs)
intermediate_output = intermediate_layer_model.predict(tf.expand_dims(all_images_rgb[0], 0))

for level in range(0,5):
    filters=intermediate_output[level][0]

    filters_num=filters.shape[-1]
    n_rows=int(math.sqrt(filters_num))

    fig,ax=plt.subplots(nrows=n_rows,ncols=n_rows,figsize=(50,50))
    step=0
    for i in range(n_rows):
        for j in range(n_rows):
            ax[i][j].imshow(filters[:,:,step],cmap='gray')
            step+=1
    # plt.savefig(f'vgg_cnn_filters_level_{1+level}.png')
plt.show()